# Классификация изображений с помощью SIFT.


Реализовать средствами OpenCV и scikit-learn следующий алгоритм генерации признаков. С помощью него решить задачу классификации CIFAR-10.


### Загружаем датасет

In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import os
import glob

In [ ]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

  0%|          | 0/135107811 [00:00<?, ?it/s]

In [ ]:
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

In [ ]:
data_dir = './data/cifar10'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'train']
['dog', 'cat', 'frog', 'airplane', 'bird', 'horse', 'ship', 'deer', 'automobile', 'truck']


## Шаг 1. Посчитать SIFT-дескрипторы для всех изображений.

In [ ]:
sift = cv.SIFT_create()

In [ ]:
def descriptors_extract(path):
  df = []
  descriptors = []
  filenames = glob.glob(path + "/*")
  for filename in filenames:
    clas_nams = glob.glob(filename + "/*")
    for clas_nam in clas_nams:
        try:
            imageID = filename[filename.rfind("/") + 1:]
            img = cv.imread(clas_nam)
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            kp, des = sift.detectAndCompute(gray, None)
            descriptors.extend(des)
            df.append((imageID,des))
        except TypeError as e:
            print(e)

  return df, descriptors


In [ ]:
path = "/content/data/cifar10/train"
train = []
train_des = []

train, train_des = descriptors_extract(path)

In [ ]:
path = "/content/data/cifar10/test"
test = []
test_des = []

test, test_des = descriptors_extract(path)

## Шаг 2. Для каждого изображения выбрать случайным образом небольшое число его дескрипторов для экономии вычислительных ресурсов.


#### Количество дескрипторов для изображений различаются, поэтому будем брать не одно и то же количество дескрипторов для каждого изображения, а определенный процент

In [ ]:
unique = []
for i in range(0, len(train)):
  _, des = train[i]
  unique.append(des.shape[0])

In [ ]:
plt.hist(unique)
plt.xlabel('descriptors')
plt.ylabel('images')
plt.show()

In [ ]:
unique_numbers = list(set(unique))
print(unique_numbers)

#### Возьмем половину всех дескрипторов

In [ ]:
print(len(train_des))
print(len(test_des))

In [ ]:
train_des = []
for i in range(0, len(train)):
  _, des = train[i]
  mask = np.random.choice([False, True], des.shape[0], p=[0.5, 0.5])
  des = des[mask]
  train_des.extend(des)

In [ ]:
test_des = []
for i in range(0, len(test)):
  _, des = test[i]
  mask = np.random.choice([False, True], des.shape[0], p=[0.5, 0.5])
  des = des[mask]
  test_des.extend(des)

In [ ]:
print(len(train_des))
print(len(test_des))

## Шаг 3. Произвести кластеризацию ВСЕХ выбранных дескрипторов по алгоритму k-means. Таким образом происходит группировка похожих дескрипторов и построение некоторого визуального словарного запаса (его размер равен количеству кластеров).


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
des_train = np.array(train_des)

In [ ]:
clusters = KMeans(n_clusters = 100)
clusters.fit_predict(des_train)
cluster = clusters.cluster_centers_

In [ ]:
clusters

## Шаг 4. Сгенерировать векторы признаков для изображений по принципу Bag-of-Words. Т.е. для каждого изображения посчитать сколько из его дескрипторов попадают в тот или иной кластер. Эти частоты попадания, собранные в вектор, и будут вектором признаков для этого изображения.


In [ ]:
import pandas as pd
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

dict = {'dog' : 0, 'automobile' : 1, 'bird' : 2, 'airplane' : 3, 'ship' : 4, 'truck' : 5, 'frog' : 6, 'horse' : 7, 'deer' : 8, 'cat' : 9}
df_train[0] = [dict[item] for item in df_train[0]]
df_test[0] = [dict[item] for item in df_test[0]]

y_train = df_train[0].to_numpy()
y_test = df_test[0].to_numpy()

In [ ]:
from scipy.cluster.vq import vq

train_features = np.zeros((len(train), 100), "float32")
for i in range(0,len(train)):
    words, distance = vq(train[i][1],cluster)
    for w in words:
        train_features[i][w] += 1

test_features = np.zeros((len(test), 100), "float32")
for i in range(0,len(test)):
    words, distance = vq(test[i][1],cluster)
    for w in words:
        test_features[i][w] += 1

## Шаг 5. Датасет готов. Для решения задачи можно использовать любой классификатор  (например, SVM).

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(train_features)
X_train = scaler.transform(train_features)

scaler = StandardScaler().fit(test_features)
X_test = scaler.transform(test_features)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear')
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

print("Score: {:.2f}%".format(score * 100))

In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)
score = xgb_clf.score(X_test, y_test)

print("Score: {:.2f}%".format(score * 100))

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier

# model = GradientBoostingClassifier()
# model.fit(X_train, y_train)
# score = model.score(X_test, y_test)

# print("Score: {:.2f}%".format(score * 100))

In [ ]:
# from xgboost import XGBClassifier

# xgb_clf = XGBClassifier()
# xgb_clf.fit(X_train, y_train)
# score = xgb_clf.score(X_train, y_train)

# print("Score: {:.2f}%".format(score * 100))

# Linear model

### Сравним один линейный слой с sift

итог:

   Score: 27.04% - sift

   Score: 37.94% - linear model

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [ ]:
trainset = CIFAR10(root='data/', download=True, transform=ToTensor())
testset = CIFAR10(root='data/', train=False, transform=ToTensor())

In [ ]:
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testLoader  = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
classes = trainset.classes
dataiter = iter(trainLoader)
images, labels =  next(dataiter)

In [ ]:
labels.type()

In [ ]:
class LinearClassifier(torch.nn.Module):
  def __init__(self, input_dim=3*32*32, output_dim=10):
    super(LinearClassifier, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self, x):
    x = self.linear(x)
    return x

In [ ]:
model  = LinearClassifier()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
all_loss = []

for epoch in range(10):
  temp_loss = []
  for images, labels in trainLoader:
    output = model(images.view(images.shape[0], -1))
    print('/////')
    print(labels)

    loss = criterion(output, labels)
    print(output)
    loss.backward()
    temp_loss.append(loss.item())
    optimizer.step()
    optimizer.zero_grad()
  all_loss.append(np.mean(temp_loss))
  print(f"Epoch: {epoch}, loss: {np.mean(temp_loss)}")

In [ ]:
correct, total = 0, 0

with torch.no_grad():
  for images, labels in testLoader:
    output = model(images.view(images.shape[0], -1))
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [ ]:
acc = 100 * correct / total
print("Score: {:.2f}%".format(acc))